In [1]:
!ls ../input/*

In [2]:
# Basic packages
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random as rd # generating random numbers
import datetime # manipulating date formats
# from pandas import datetime
# Viz
import matplotlib.pyplot as plt # basic plotting
import seaborn as sns # for prettier plots



# TIME SERIES
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.stattools import adfuller, acf, pacf,arma_order_select_ic
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs


# settings
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Import all of them 
sales_train=pd.read_csv("../input/sales_train.csv")

# settings
import warnings
warnings.filterwarnings("ignore")

item_categories=pd.read_csv("../input/item_categories.csv")
items=pd.read_csv("../input/items.csv")
sample_submission=pd.read_csv("../input/sample_submission.csv")
shops=pd.read_csv("../input/shops.csv")
test=pd.read_csv("../input/test.csv")

In [4]:
# check every data file
print('==========================items categories============================')
item_categories.info()
item_categories.columns
print('===============================items===================================')
items.info()
print(items.columns)
print('===============================test====================================')
test.info()
print(test.columns)
print('=================================sales=================================')
sales_train.info()
print(sales_train.columns)
print('=================================shops===============================')
shops.info()
print(shops.columns)

In [5]:
# agregate sales to monthly sale, the count of each item for each shop
mon_sales_eashop = sales_train.groupby(["date_block_num","shop_id","item_id"])["item_price","item_cnt_day"].agg({"item_price":"mean","item_cnt_day":"sum"})
print(mon_sales_eashop.head(10))

In [6]:
# to have a general understand to the data
x=items.groupby(['item_category_id']).count()
x=x.sort_values(by='item_id',ascending=False)
x=x.iloc[0:20].reset_index()
print(x)
# #plot
plt.figure(figsize=(12,6))
ax= sns.barplot(x.item_category_id, x.item_id, alpha=0.3)
plt.title("Items per Category")
plt.ylabel('# of items', fontsize=12)
plt.xlabel('Category', fontsize=12)
plt.show()

In [7]:
#the general trend of sales value 
monthList = sales_train.date_block_num.unique()
salesDict = {}

for item in monthList:
    salesDict[item] = 0

for item in sales_train.itertuples():
    salesDict[item.date_block_num] =  round(salesDict[item.date_block_num] + item.item_cnt_day * item.item_price, 2)

sale_values = pd.Series(list(salesDict.values()))

plt.figure(figsize=(9,9))
plt.subplot(311)
plt.title('Original')
plt.xlabel('Time/mon')
plt.ylabel('Sales/$')
plt.plot(sale_values)

In [8]:
plt.figure(figsize=(8,4))
plt.plot(sale_values.rolling(window=12,center=False).mean(),label='Rolling Mean');
plt.plot(sale_values.rolling(window=12,center=False).std(),label='Rolling sd');
plt.legend();

In [9]:
# divide a series to level, trend, seasonality, and noise components, 
# to check "additive" and "multiplicative" model
import statsmodels.api as sm
# multiplicative
print(sale_values.values)
res = sm.tsa.seasonal_decompose(sale_values.values,freq=12,model="multiplicative")
#plt.figure(figsize=(16,12))
fig = res.plot()
#fig.show()

In [10]:
import statsmodels.api as sm
# multiplicative
print(sale_values.values)
res = sm.tsa.seasonal_decompose(sale_values.values, freq=12, model="additive")
#plt.figure(figsize=(16,12))
fig = res.plot()
#fig.show()

In [11]:
# Stationarity tests: ADF test to show whether the time serious is stable

def ADF_test(timeseries):
    stationary = True
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:1], index=['Test Statistic','p-value'])
    for key,value in dftest[4].items():
        if(key == "5%"): 
            dfoutput['Critical Value (%s)'%key] = value
    if dfoutput['Test Statistic'] < dfoutput['Critical Value (5%)']:
        stationary = True
    else: stationary = False 
    return stationary
    

t = ADF_test(sale_values)
t

In [12]:
# As test statistic < critical Value(5%), p value < 0.05, assume the series is stationary. 
# Test whether this is non-white noise sequence
from statsmodels.stats.diagnostic import acorr_ljungbox
acorr_ljungbox(sale_values, lags = [i for i in range(0,13)])

In [13]:
# as pvalue is much less than stat and close to 0, this is not a white noise
# define a function to display ACF, PACF, PP plot and QQ plot:
def tsplot(y, lags=None, figsize=(10, 8), title=''):
    if not isinstance(y, pd.Series):
        y = pd.Series(y)   
        fig = plt.figure(figsize=figsize)
        layout = (3, 2)
        ts_ax = plt.subplot2grid(layout, (0, 0), colspan=2)
        acf_ax = plt.subplot2grid(layout, (1, 0))
        pacf_ax = plt.subplot2grid(layout, (1, 1))
        qq_ax = plt.subplot2grid(layout, (2, 0))
        pp_ax = plt.subplot2grid(layout, (2, 1))
        
        y.plot(ax=ts_ax)
        ts_ax.set_title(title)
        smt.graphics.plot_acf(y, lags=lags, ax=acf_ax, alpha=0.5)
        smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax, alpha=0.5)
        sm.qqplot(y, line='s', ax=qq_ax)
        qq_ax.set_title('QQ Plot')        
        scs.probplot(y, sparams=(y.mean(), y.std()), plot=pp_ax)

        plt.tight_layout()
    return 

In [14]:
# Simulate an AR(1) process with alpha = 0.5
np.random.seed(1)
n_samples = int(1000)
a = 0.5
x = E = np.random.normal(size=n_samples)

for t in range(n_samples):
    x[t] = a*x[t-1] + E[t]
limit=12    
_ = tsplot(x, lags=limit,title="AR(1)process")

In [15]:
# Simulate an AR(2) process

n = int(1000)
alphas = np.array([0.46, 0.3])
betas = np.array([0])

# Python requires us to specify the zero-lag value which is 1
# Also note that the alphas for the AR model must be negated
# We also set the betas for the MA equal to 0 for an AR(p) model

ar = np.r_[1, -alphas]
ma = np.r_[1, betas]

ar2 = smt.arma_generate_sample(ar=ar, ma=ma, nsample=n) 
_ = tsplot(ar2, lags=12,title="AR(2) process")

#AR(1) process has ACF tailing out and PACF cutting off at lag=1 
#AR(2) process has ACF tailing out and PACF cutting off at lag=2

In [16]:
# Simulate an MA(1) process
# Set alpha for the AR equal to 0

n = int(1000)
alphas = np.array([0])
betas = np.array([0.65])


ar = np.r_[1, -alphas]
ma = np.r_[1, betas]

ar2 = smt.arma_generate_sample(ar=ar, ma=ma, nsample=n) 
_ = tsplot(ar2, lags=12,title="MA(1) process")

In [17]:
# Simulate an MA(2) process
# Set alpha for the AR equal to 0


n = int(1000)
alphas = np.array([0])
betas = np.array([0.7,0.2])


ar = np.r_[1, -alphas]
ma = np.r_[1, betas]

MA2 = smt.arma_generate_sample(ar=ar, ma=ma, nsample=n) 
_ = tsplot(MA2, lags=12,title="MA(2) process")

#### MA(1) process has ACF cut off at lag=1 and PACF tailing out 
#### MA(2) process has ACF cut off at lag=2 and PACF tailing out 

In [18]:
max_lag = 12

n = int(5000) # lots of samples to help estimates
burn = int(n/10) # number of samples to discard before fit

alphas = np.array([0.8, -0.65])
betas = np.array([0.5, -0.7])
ar = np.r_[1, -alphas]
ma = np.r_[1, betas]

arma22 = smt.arma_generate_sample(ar=ar, ma=ma, nsample=n, burnin=burn)
_ = tsplot(arma22, lags=max_lag,title="ARMA(2,2) process")

In [19]:
# pick best order by aic 
# smallest aic value wins
best_aic = np.inf 
best_order = None
best_mdl = None

rng = range(5)
for i in rng:
    for j in rng:
        try:
            tmp_mdl = smt.ARMA(arma22, order=(i, j)).fit(method='mle', trend='nc')
            tmp_aic = tmp_mdl.aic
            if tmp_aic < best_aic:
                best_aic = tmp_aic
                best_order = (i, j)
                best_mdl = tmp_mdl
        except: continue

print('aic: {:6.5f} | order: {}'.format(best_aic, best_order))


### We identified the order of the simulated process as ARMA(2,2). 

#### Lets use it for the sales time-series.


In [20]:
# Simply use best_mdl.predict() to predict the next values

In [21]:
# adding the dates to the Time-series as index
ts=sale_values
ts.index=pd.date_range(start = '2013-01-01',end='2015-10-01', freq = 'MS')
ts=ts.reset_index()
ts.head()

In [22]:
from fbprophet import Prophet
#prophet reqiures a pandas df at the below config 
# ( date column named as DS and the value column as Y)
ts.columns=['ds','y']
model = Prophet( yearly_seasonality=True) #instantiate Prophet with only yearly seasonality as our data is monthly 
model.fit(ts) #fit the model with your dataframe

In [23]:
# predict for five months in the furure and MS - month start is the frequency
future = model.make_future_dataframe(periods = 5, freq = 'MS')  
# now lets make the forecasts
forecast = model.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [24]:
model.plot(forecast)

In [25]:
model.plot_components(forecast)